In [1]:
import json

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np

2022-08-26 15:21:49.400797: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
def _load_json(json_file):
    with open(json_file) as f:
        data = json.load(f)
    return data

def load_results_to_df(json_results_patern):
    if isinstance(json_results_patern, list):
        json_files = []
        for patern in json_results_patern:
            json_files += tf.io.gfile.glob(patern)
    else:
        json_files = tf.io.gfile.glob(json_results_patern)
    
    results = [_load_json(jfile) for jfile in json_files]

    return pd.DataFrame(results)

def format_results(df):
    df['f_1^m'] = df['f1_score_cls'].apply(np.mean).round(3)
    df = df.drop(['f1_score_cls'], axis=1)
    
    df = pd.concat([df.drop(['acc_bins'], axis=1),
                      df['acc_bins'].apply(pd.Series).add_prefix('acc_')],
                     axis=1)
    df = df[['dataset', 'model', 'training_setup', 'acc_1', 'acc_2', 'acc_3', 'acc_4', 'acc_all', 'f_1^m']]
    df[['acc_1', 'acc_2', 'acc_3', 'acc_4', 'acc_all']] = (df[['acc_1', 'acc_2', 'acc_3', 'acc_4', 'acc_all']]*100).round(2)

    return df

def filter_results(df, dataset, column_list):
    df = df[(df.dataset==dataset)]
    df = df[['model', 'training_setup'] + column_list]
    df = df.set_index(['model','training_setup'])
    columns = [(dataset, col) for col in list(df.columns)]
    df.columns=pd.MultiIndex.from_tuples(columns)
    df = df.copy()
    
    return df

In [3]:
results_json = ['/data/fagner/training/bags_paper/results/exp1/*_results.json',
                '/data/fagner/training/bags_paper/checkpoints/*imagenet*/*_results.json',
                '/data/fagner/training/bags_paper/checkpoints/*_ssb_*/*_results.json']
results = load_results_to_df(results_json)
results = format_results(results)

training_setup_sort = {'repre': 0,
                       'imagenet': 1,
                       'crt': 2,
                       'imagenetcrt': 3,
                       'cbfocal': 4,
                       'imagenetcbfocal': 5,
                       'bags': 6,
                       'imagenetbags': 7,
                       'ssb': 8,
                       'resnet50': 0,
                       'mbnetv3': 1,
                       'effv2b2': 2,
                       'swin-s': 3}

results = results.sort_values(by=['model', 'training_setup'], key=lambda x: x.map(training_setup_sort))

In [4]:
results = results[results.training_setup.isin(['repre', 'crt', 'cbfocal', 'bags', 'ssb'])]

In [5]:
wcs_results = filter_results(results, 'wcs', ['acc_1', 'acc_2', 'acc_3', 'acc_4', 'acc_all', 'f_1^m'])
wcs_results

wcs                                    
                         acc_1  acc_2  acc_3  acc_4 acc_all  f_1^m
model    training_setup                                           
resnet50 repre            0.00  13.05  32.06  85.02   81.53  0.289
         crt             16.25  28.94  48.98  75.38   73.47  0.322
         cbfocal          5.00  14.42  40.61  83.41   80.43  0.316
         bags            16.25  14.19  34.84  84.43   81.14  0.283
         ssb             15.00  21.79  40.97  84.85   81.91  0.343
mbnetv3  repre            0.00  11.92  36.52  84.24   80.98  0.287
         crt              7.50  28.26  46.85  76.71   74.61  0.292
         cbfocal          0.00   9.19  36.98  79.41   76.42  0.276
         bags            10.00  10.44  28.68  84.09   80.48  0.248
         ssb              1.25  20.32  40.43  83.99   81.04  0.319
effv2b2  repre            0.00  10.22  32.31  86.35   82.75  0.278
         crt             10.00  22.36  50.34  78.24   76.12  0.343
         cbfocal          0.00   4.20  35.55  87.20   83.62  0.283
         bags             1.25   7.15  32.99  87.01   83.37  0.277
         ssb              7.50  18.39  42.39  86.18   83.17  0.355
swin-s   repre            8.75  37.12  62.99  90.86   88.76  0.526
         crt             16.25  44.84  67.22  83.49   82.13  0.477
         cbfocal          0.00  25.09  56.57  90.32   87.78  0.452
         bags             6.25  21.68  52.33  90.08   87.32  0.409
         ssb             16.25  40.18  64.91  90.65   88.69  0.523

In [6]:
serengeti_results = filter_results(results, 'serengeti', ['acc_2', 'acc_3', 'acc_4', 'acc_all', 'f_1^m'])
serengeti_results

serengeti                             
                            acc_2  acc_3  acc_4 acc_all  f_1^m
model    training_setup                                       
resnet50 repre               2.90  44.82  92.96   92.79  0.522
         crt                 6.52  51.58  87.29   87.16  0.396
         cbfocal             0.72  38.66  92.88   92.69  0.492
         bags                9.42  41.27  93.12   92.94  0.499
         ssb                 5.07  48.16  92.91   92.75  0.518
mbnetv3  repre               2.17  29.78  92.73   92.52  0.457
         crt                 4.35  43.95  87.62   87.47  0.396
         cbfocal             0.72  29.78  92.44   92.23  0.460
         bags                5.07  28.78  92.19   91.97  0.432
         ssb                 3.62  36.97  92.72   92.53  0.478
effv2b2  repre               2.90  53.71  93.82   93.68  0.575
         crt                 6.52  59.25  90.46   90.35  0.501
         cbfocal             0.00  46.51  94.51   94.34  0.560
         bags                7.25  51.45  94.77   94.62  0.563
         ssb                 5.80  55.96  93.81   93.67  0.581
swin-s   repre              10.87  60.42  95.09   94.97  0.643
         crt                13.04  64.85  89.08   88.99  0.464
         cbfocal             4.35  53.71  94.97   94.82  0.616
         bags               15.22  52.32  94.81   94.67  0.583
         ssb                11.59  61.94  95.05   94.93  0.626

In [7]:
caltech_results = filter_results(results, 'caltech', [ 'acc_2', 'acc_4', 'acc_all', 'f_1^m'])
caltech_results

caltech                      
                          acc_2  acc_4 acc_all  f_1^m
model    training_setup                              
resnet50 repre             0.00  44.95   42.19  0.301
         crt               0.00  39.59   37.16  0.297
         cbfocal           0.00  44.47   41.74  0.314
         bags              1.56  48.04   45.19  0.326
         ssb               0.03  46.09   43.26  0.310
mbnetv3  repre             0.00  46.14   43.31  0.319
         crt               0.16  37.76   35.45  0.284
         cbfocal           0.00  41.27   38.74  0.302
         bags              0.00  43.96   41.26  0.304
         ssb               0.11  45.93   43.12  0.323
effv2b2  repre             0.08  56.36   52.91  0.338
         crt               0.00  51.15   48.01  0.334
         cbfocal           0.00  57.36   53.83  0.353
         bags              0.26  58.88   55.28  0.358
         ssb               0.08  59.82   56.15  0.372
swin-s   repre             4.17  68.94   64.96  0.520
         crt               1.08  64.95   61.02  0.460
         cbfocal           3.78  69.61   65.57  0.484
         bags              3.67  66.01   62.18  0.464
         ssb               1.08  67.57   63.49  0.497

In [8]:
wellington_results = filter_results(results, 'wellington', ['acc_2', 'acc_3', 'acc_4', 'acc_all', 'f_1^m'])
wellington_results

wellington                             
                             acc_2  acc_3  acc_4 acc_all  f_1^m
model    training_setup                                        
resnet50 repre                 0.0   4.58  75.66   72.23  0.309
         crt                   0.0  10.65  70.91   68.01  0.315
         cbfocal               0.0   4.84  75.12   71.73  0.312
         bags                  0.0  25.92  75.00   72.64  0.319
         ssb                   0.0   5.51  75.62   72.24  0.312
mbnetv3  repre                 0.0   5.70  75.65   72.28  0.326
         crt                   0.0  21.68  70.45   68.10  0.313
         cbfocal               0.0   4.58  74.94   71.55  0.313
         bags                  0.0  23.41  74.02   71.58  0.292
         ssb                   0.0  13.54  75.34   72.36  0.326
effv2b2  repre                 0.0   5.70  77.11   73.67  0.344
         crt                   0.0   8.06  74.14   70.95  0.340
         cbfocal               0.0   4.09  77.31   73.78  0.346
         bags                  0.0  30.05  76.00   73.79  0.350
         ssb                   0.0   6.75  77.01   73.62  0.340
swin-s   repre                 0.0   7.61  78.94   75.51  0.427
         crt                   0.0  13.09  76.54   73.48  0.418
         cbfocal               0.0   5.44  78.49   74.98  0.413
         bags                  0.0  19.65  78.62   75.78  0.376
         ssb                   0.0   7.16  78.86   75.41  0.424

In [9]:
result = pd.concat([wcs_results, serengeti_results, caltech_results, wellington_results], axis=1)
result

wcs                                     serengeti  \
                         acc_1  acc_2  acc_3  acc_4 acc_all  f_1^m     acc_2   
model    training_setup                                                        
resnet50 repre            0.00  13.05  32.06  85.02   81.53  0.289      2.90   
         crt             16.25  28.94  48.98  75.38   73.47  0.322      6.52   
         cbfocal          5.00  14.42  40.61  83.41   80.43  0.316      0.72   
         bags            16.25  14.19  34.84  84.43   81.14  0.283      9.42   
         ssb             15.00  21.79  40.97  84.85   81.91  0.343      5.07   
mbnetv3  repre            0.00  11.92  36.52  84.24   80.98  0.287      2.17   
         crt              7.50  28.26  46.85  76.71   74.61  0.292      4.35   
         cbfocal          0.00   9.19  36.98  79.41   76.42  0.276      0.72   
         bags            10.00  10.44  28.68  84.09   80.48  0.248      5.07   
         ssb              1.25  20.32  40.43  83.99   81.04  0.319      3.62   
effv2b2  repre            0.00  10.22  32.31  86.35   82.75  0.278      2.90   
         crt             10.00  22.36  50.34  78.24   76.12  0.343      6.52   
         cbfocal          0.00   4.20  35.55  87.20   83.62  0.283      0.00   
         bags             1.25   7.15  32.99  87.01   83.37  0.277      7.25   
         ssb              7.50  18.39  42.39  86.18   83.17  0.355      5.80   
swin-s   repre            8.75  37.12  62.99  90.86   88.76  0.526     10.87   
         crt             16.25  44.84  67.22  83.49   82.13  0.477     13.04   
         cbfocal          0.00  25.09  56.57  90.32   87.78  0.452      4.35   
         bags             6.25  21.68  52.33  90.08   87.32  0.409     15.22   
         ssb             16.25  40.18  64.91  90.65   88.69  0.523     11.59   

                                                     caltech                 \
                         acc_3  acc_4 acc_all  f_1^m   acc_2  acc_4 acc_all   
model    training_setup                                                       
resnet50 repre           44.82  92.96   92.79  0.522    0.00  44.95   42.19   
         crt             51.58  87.29   87.16  0.396    0.00  39.59   37.16   
         cbfocal         38.66  92.88   92.69  0.492    0.00  44.47   41.74   
         bags            41.27  93.12   92.94  0.499    1.56  48.04   45.19   
         ssb             48.16  92.91   92.75  0.518    0.03  46.09   43.26   
mbnetv3  repre           29.78  92.73   92.52  0.457    0.00  46.14   43.31   
         crt             43.95  87.62   87.47  0.396    0.16  37.76   35.45   
         cbfocal         29.78  92.44   92.23  0.460    0.00  41.27   38.74   
         bags            28.78  92.19   91.97  0.432    0.00  43.96   41.26   
         ssb             36.97  92.72   92.53  0.478    0.11  45.93   43.12   
effv2b2  repre           53.71  93.82   93.68  0.575    0.08  56.36   52.91   
         crt             59.25  90.46   90.35  0.501    0.00  51.15   48.01   
         cbfocal         46.51  94.51   94.34  0.560    0.00  57.36   53.83   
         bags            51.45  94.77   94.62  0.563    0.26  58.88   55.28   
         ssb             55.96  93.81   93.67  0.581    0.08  59.82   56.15   
swin-s   repre           60.42  95.09   94.97  0.643    4.17  68.94   64.96   
         crt             64.85  89.08   88.99  0.464    1.08  64.95   61.02   
         cbfocal         53.71  94.97   94.82  0.616    3.78  69.61   65.57   
         bags            52.32  94.81   94.67  0.583    3.67  66.01   62.18   
         ssb             61.94  95.05   94.93  0.626    1.08  67.57   63.49   

                               wellington                               
                         f_1^m      acc_2  acc_3  acc_4 acc_all  f_1^m  
model    training_setup                                                 
resnet50 repre           0.301        0.0   4.58  75.66   72.23  0.309  
         crt             0.297        0.0  10.65  70.91   68.01  0.315  
   